In [1]:
#instantiate cursor and load libraries
import json
import pandas as pd
import requests
import mysql.connector
import config
import time


In [2]:
#specify database for connection
db = mysql.connector.connect(
    host = config.host,
    user = config.user,
    password = config.password,
    database = "Movies")

cursor = db.cursor()

In [3]:
#create database
from mysql.connector import errorcode
db_name = 'Movies'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [4]:
import requests
from bs4 import BeautifulSoup as BS

In [120]:
#first page

page1 = requests.get('https://www.imdb.com/search/title/?release_date=2019-01-01,2019-10-31&sort=boxoffice_gross_us,desc&start=251&ref_=adv_nxt')
soup1 = BS(page1.content, 'html.parser')
soup1.prettify()
info1 = soup1.find_all(id = 'main')
movie1 = info1[0].find_all(class_='lister-item mode-advanced')

In [121]:
# insert_movies(make_df(movie1))

In [89]:
def get_score(movie):
        metascores = []
        for score in movie:
            metascore1 = score.find(class_="metascore")
            if metascore1 == None:
                metascore = 'Null'
                metascores.append(metascore)
            else:
                metascores.append(int(metascore1.get_text(strip=True)))
        return metascores
# len(get_score(movie))

In [61]:
def find_fk(movie):
    ids = []
    for m in movie:
        id = m.find (class_="starBarWidget")['id']
        ids.append(id)
    return ids

# find_fk(movie1)

In [8]:
def get_titles(movie):

    titles = []
    for one in movie:
        title = one.find('img')['alt']
        titles.append(title) 
    return titles

# len(get_titles(movie))

In [85]:
def get_stars(movie):
    stars = []
    for one in movie:
        try:
            stars.append(float(one.find(class_='inline-block ratings-imdb-rating')['data-value']))
        except:
            stars.append('Null')
#         stars.append(float(x['data-value']))
    return stars

# len(get_stars(movie))

In [118]:
def get_year(movie):
    
    years = []
    for one in movie:
        try:
            year = one.find(class_="lister-item-year text-muted unbold").get_text()
            parsed_year = year[-5:-1]
            years.append(int(parsed_year))
        except:
            years.append(2019)
    return years
# get_year(movie1)

In [110]:
def get_genre(movie):
    genres = []
    for m in movie:
        genre = m.find(class_='genre')
        try:
            genre1 = str.split(genre.get_text(strip=True), sep = ',')[0]
        except:
            genre1 = genre
        genres.append(genre1)
    return genres
# len(get_genre(movie1))

In [29]:
def get_gross(movie):
    gross_list = []
    
    for m in movie:
        gross = m.find(class_='sort-num_votes-visible')
        parsed_gross = gross.get_text(strip=True)
        parsed_gross2 = str.split(parsed_gross, sep='|')
       # print(parsed_gross2[1])
        if 'Gross' in parsed_gross2[0]:
            parsed_gross2 = parsed_gross2[0]
#             print(parsed_gross2[0])
            parsed_gross3 = str.split(parsed_gross2, sep=':')
#             print(parsed_gross3)
            parsed_gross4 = str.split(parsed_gross3[1], sep='.')[0]
            final_gross = str.split(parsed_gross4, sep='$')[1]
            gross_list.append(int(final_gross))
#             print(parsed_gross2)
        else:
            try:       
                parsed_gross3 = str.split(parsed_gross2[1], sep=':')
                parsed_gross4 = str.split(parsed_gross3[1], sep='.')[0]
                final_gross = str.split(parsed_gross4, sep='$')[1]
                gross_list.append(int(final_gross))
            except: 
                gross_list.append(0)
    
    return gross_list

# len(get_gross(movie))

In [133]:
gross = movie1[0].find(class_='sort-num_votes-visible').get_text(strip=True)
gross

'Votes:22|Gross:$0.01M'

In [44]:
#create table
DB_NAME = 'Movies'

create_query = """
      CREATE TABLE imdb_movies
    (
      imdb_id varchar(24),
      title varchar(50),
      year INT(4),
      star_rating FLOAT(3),
      metascore INT(5),
      gross INT(5),
      genre varchar(30),
      CONSTRAINT primary_key PRIMARY KEY(imdb_id)
      )
    """
cursor.execute(create_query)


In [28]:
#make into DF
def make_df(movie):
    all_movies_dataframe = pd.DataFrame({
        "imdb_id": find_fk(movie),
        "title": get_titles(movie),
        "year": get_year(movie),
        "star_rating": get_stars(movie),
        "metascore": get_score(movie),
        "gross": get_gross(movie),
        "genre": get_genre(movie)
    })
    return all_movies_dataframe


In [45]:
#uses DF to insert into mySQL
def insert_movies(dataframe):
    all_movies_tuples = list(dataframe.itertuples(index=False, name=None))
    insert_statement = """INSERT INTO imdb_movies (imdb_id, title, year, star_rating, metascore, gross, genre) 
            VALUES (%s, %s, %s, %s, %s, %s, %s);"""
    for movie in all_movies_tuples:
        
        cursor.execute(insert_statement, movie)
        print(movie)
    
#     cursor.executemany(insert_statement, all_movies_tuples)
    db.commit()


In [130]:
var = 251
while var < 750: 
    page = requests.get(f"https://www.imdb.com/search/title/?release_date=2019-01-01,2019-10-31&sort=boxoffice_gross_us,desc&start={var}&ref_=adv_nxt")
    soup = BS(page.content, 'html.parser')
    soup.prettify()
    info = soup.find_all(id = 'main')
    movie = info[0].find_all(class_='lister-item mode-advanced')
    movie_df = make_df(movie)
    insert_movies(movie_df)
    # #     print(movie_df)
    var +=50


AttributeError: 'NoneType' object has no attribute 'get_text'